In [5]:
import re
import nlp516
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


## Dataset

In [6]:
dataset = nlp516.data.PublicTrialRaw()

In [7]:
print('Original: {}'.format(dataset.en.iloc[1].text))
print('Tokens: {}'.format(en.iloc[1].text))

Original: @ArianasBotch Ok if you fucking said leave block me. But dm me first I'm gonna kick your ass. Shut the fuck up you https://t.co/6BXkfxMEf9


NameError: name 'en' is not defined

In [3]:
tokenizer_map = nlp516.data.Tokenizer('english')
stopwords_map = nlp516.data.RemoveStopWords('english')
stemmer_map = nlp516.data.Stemmer('english')


en = nlp516.data.map_column(dataset.en, 'text', nlp516.data.remove_urls_map)
#en = nlp516.data.map_column(dataset.en, 'text', nlp516.data.casual_tokenize_map)
en = nlp516.data.map_column(en, 'text', tokenizer_map)
en = nlp516.data.map_column(en, 'text', nlp516.data.user_camelcase_map)
en = nlp516.data.map_column(en, 'text', nlp516.data.hashtag_camelcase_map)
en = nlp516.data.map_column(en, 'text', nlp516.data.to_lowercase)
en = nlp516.data.map_column(en, 'text', stopwords_map)
en = nlp516.data.map_column(en, 'text', stemmer_map)
en = nlp516.data.map_column(en, 'text', nlp516.data.remove_words_with_numbers)


# Preprocessing

In [1]:
print('Original: {}'.format(dataset.en.iloc[25].text))
print('Tokens: {}'.format(en.iloc[25].text))

NameError: name 'dataset' is not defined

In [ ]:
print('Original: {}'.format(dataset.en.iloc[26].text))
print('Tokens: {}'.format(en.iloc[26].text))

## Feature representation

In [7]:
class Vectorizer(object):
    def __init__(self, n_features):
        self.n_features = n_features
        
    def id2word(self, id):
        vocabulary_inv = {value:key for key, value in self.vectorizer.vocabulary_.items()}
        return vocabulary_inv[id]

    def fit(self, data):
        self.train = data
        self.vectorizer = CountVectorizer()
        self.x = self.vectorizer.fit_transform(data)
        #count = {id: c for c in self.x.toarray().sum(0)}
        self.feature_id = list(filter(lambda n: np.sum(self.x.toarray(), axis=0)[n]>2, range(self.x.shape[-1])))
        
        
    def transform(self, data):
        x = self.vectorizer.transform(data)
        return x[:, self.feature_id]
        



## Train

In [85]:
import sklearn.linear_model

In [101]:
def random_shuffle(data):
    idx = np.arange(en.shape[0])
    shuffle = np.random.shuffle(idx)
    return data.iloc[idx, :]
dataset = random_shuffle(en)
train = dataset[:50]
test = dataset[50:]

In [102]:
class MlModel(object):
    @staticmethod
    def get_corpus(dataset):
        return [' '.join(sample) for sample in dataset]
    
    def _fit_vectorizer(self, x):
        corpus = self.get_corpus(x)
        self.vectorizer = Vectorizer(50)
        self.vectorizer.fit(corpus)
        
    
    def _fit_model(self, x, y):
        corpus = self.get_corpus(x)
        train_x = self.vectorizer.transform(corpus)
        train_y = y
        self.model = sklearn.linear_model.LogisticRegression().fit(train_x, train_y)
        
    def fit(self, x, y):
        self._fit_vectorizer(x)
        self._fit_model(x, y)
        print('score: {}'.format(self.score(x, y)))
    
    def predict(self, x):
        corpus = self.get_corpus(x)
        test_x = self.vectorizer.transform(corpus)
        return self.model.predict(test_x)
    
    def score(self, x, y):
        corpus = self.get_corpus(x)
        test_x = self.vectorizer.transform(corpus)
        test_y = self.model.predict(test_x)
        return np.mean(test_y == y)

In [103]:
model = MlModel()
model.fit(train.text, train.HS)
print('test: {}'.format(model.score(test.text, test.HS)))

score: 0.88
test: 0.7


In [104]:
model = MlModel()
model.fit(train.text, train.TR)
print('test: {}'.format(model.score(test.text, test.TR)))

score: 0.86
test: 0.94


In [105]:
model = MlModel()
model.fit(train.text, train.AG)
print('test: {}'.format(model.score(test.text, test.AG)))

score: 0.86
test: 0.84


In [84]:
#[model.vectorizer.id2word(id) for id in model.vectorizer.feature_id]